# Goldstein-price function

**Contributed by**: Benoît Legat

In this example, we consider the minimization of the [Goldstein-price function](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

In [1]:
using SumOfSquares
using DynamicPolynomials

Create *symbolic* variables (not JuMP *decision* variables)

In [2]:
@polyvar x[1:2]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂],)

To use Sum-of-Squares Programming, we first need to pick an SDP solver,
see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [3]:
import Clarabel
using Dualization
model = SOSModel(dual_optimizer(Clarabel.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Dual model with Clarabel attached

Create a JuMP decision variable for the lower bound

In [4]:
@variable(model, γ)

γ

`f(x)` is the Goldstein-Price function

In [5]:
f1 = x[1] + x[2] + 1
f2 = 19 - 14*x[1] + 3*x[1]^2 - 14*x[2] + 6*x[1]*x[2] + 3*x[2]^2
f3 = 2*x[1] - 3*x[2]
f4 = 18 - 32*x[1] + 12*x[1]^2 + 48*x[2] - 36*x[1]*x[2] + 27*x[2]^2
f = (1 + f1^2*f2) * (30 + f3^2*f4)

600 + 720x₂ + 720x₁ + 3060x₂² - 4680x₁x₂ + 1260x₁² + 12288x₂³ - 19296x₁x₂² + 7344x₁²x₂ - 1072x₁³ + 14346x₂⁴ - 23616x₁x₂³ + 7776x₁²x₂² + 5784x₁³x₂ - 2454x₁⁴ + 1944x₂⁵ - 11880x₁x₂⁴ + 5040x₁²x₂³ + 9840x₁³x₂² - 7680x₁⁴x₂ + 1344x₁⁵ - 4428x₂⁶ - 1188x₁x₂⁵ + 8730x₁²x₂⁴ + 1240x₁³x₂³ - 5370x₁⁴x₂² - 168x₁⁵x₂ + 952x₁⁶ - 648x₂⁷ + 1944x₁x₂⁶ + 3672x₁²x₂⁵ - 3480x₁³x₂⁴ - 4080x₁⁴x₂³ + 2592x₁⁵x₂² + 1344x₁⁶x₂ - 768x₁⁷ + 729x₂⁸ + 972x₁x₂⁷ - 1458x₁²x₂⁶ - 1836x₁³x₂⁵ + 1305x₁⁴x₂⁴ + 1224x₁⁵x₂³ - 648x₁⁶x₂² - 288x₁⁷x₂ + 144x₁⁸

Constraints `f(x) - γ` to be a sum of squares

In [6]:
con_ref = @constraint(model, f >= γ)
@objective(model, Max, γ)
optimize!(model)

-------------------------------------------------------------
           Clarabel.jl v0.7.1  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 45
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 121
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : PSDTriangle = 1,  numel = 120

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-05, max_scale = 1.0e+05
               max iter = 10

iter    pcost        dcost       ga

The lower bound found is 3

In [7]:
solution_summary(model)

* Solver : Dual model with Clarabel attached

* Status
  Result count       : 1
  Termination status : ALMOST_OPTIMAL
  Message from the solver:
  "ALMOST_SOLVED"

* Candidate solution (result #1)
  Primal status      : NEARLY_FEASIBLE_POINT
  Dual status        : NEARLY_FEASIBLE_POINT
  Objective value    : 3.00006e+00
  Dual objective value : 3.00005e+00

* Work counters
  Solve time (sec)   : 2.99624e-02
  Barrier iterations : 18


The moment matrix is as follows, we can already see the global minimizer
`[0, -1]` from the entries `(2, 1)` and `(3, 1)`.
This heuristic way to obtain solutions to the polynomial optimization problem
is suggested in [Laurent2008; (6.15)](@cite).

In [8]:
ν = moment_matrix(con_ref)

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3, x[2]^4, x[1]*x[2]^3, x[1]^2*x[2]^2, x[1]^3*x[2], x[1]^4])
And entries in a 15×15 SymMatrix{Float64}:
  1.0000000018573474     -0.999998184576739      …   3.5962086853253903e-6
 -0.999998184576739       0.999997150198852          1.0531832701061796e-6
 -1.2636567926751192e-6   1.4804132265158204e-6      6.267497079311888e-6
  0.9999971886457574     -0.9999960645215564         0.00021144386295502015
  1.4653553927555777e-6  -1.1547395566676937e-6      0.00021277355946083603
  1.187350682495253e-6    1.605777819548949e-7   …   0.0004346101519721536
 -0.9999960345776464      0.9999952039300222        -0.00037037588687171245
 -1.1433741839080068e-6   1.273723901361668e-6      -0.00041077802563794515
  1.5687335193614228e-7   3.467245543475798e-7      -0.00022841391643848772
  1.2660969071438837e-6   8.152487714237158e-7      -6.123905166790361e-5
  0.999995

Many entries of the matrix actually have the same moment.
We can obtain the following list of these moments without duplicates
(ignoring when difference of entries representing the same moments is below `1e-5`)

In [9]:
μ = measure(ν, atol = 1e-5)

Measure{Float64, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}([1.0000000018573474, -0.999998184576739, -1.2636567926751192e-6, 0.9999971886457574, 1.4653553927555777e-6, 1.187350682495253e-6, -0.9999960345776464, -1.1433741839080068e-6, 1.5687335193614228e-7, 1.2660969071438837e-6  …  -6.123905166790361e-5, 1.270218769208648, -0.19058213646104005, 0.3086716525700899, -0.1318706321712997, 0.39570337503546366, -0.0010509740296115082, 0.5917596920647119, 0.2928164690573049, 1.0338233525191614], DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[1, x₂, x₁, x₂², x₁x₂, x₁², x₂³, x₁x₂², x₁²x₂, x₁³  …  x₁⁷, x

The truncated moment matrix can then be obtained as follows

In [10]:
ν_truncated = moment_matrix(μ, monomials(x, 0:3))

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3])
And entries in a 10×10 SymMatrix{Float64}:
  1.0000000018573474     -0.999998184576739      …  1.2660969071438837e-6
 -0.999998184576739       0.9999971886457574        8.265186088348277e-7
 -1.2636567926751192e-6   1.4653553927555777e-6     3.5962086853253903e-6
  0.9999971886457574     -0.9999960345776464        6.000631118417262e-8
  1.4653553927555777e-6  -1.1433741839080068e-6     1.0531832701061796e-6
  1.187350682495253e-6    1.5687335193614228e-7  …  6.267497079311888e-6
 -0.9999960345776464      0.9999951935941798        7.041812532662351e-5
 -1.1433741839080068e-6   1.271793213373499e-6      0.00021144386295502015
  1.5687335193614228e-7   3.573504479057564e-7      0.00021277355946083603
  1.2660969071438837e-6   8.265186088348277e-7      0.0004346101519721536

Let's check if the flatness property is satisfied.
The rank of `ν_truncated` seems to be 1:

In [11]:
using LinearAlgebra
LinearAlgebra.svdvals(Matrix(ν_truncated.Q))
LinearAlgebra.rank(Matrix(ν_truncated.Q), rtol = 1e-3)
svdvals(Matrix(ν_truncated.Q))

10-element Vector{Float64}:
 4.00000395428376
 0.0006683576478051575
 0.00014945001542051055
 7.258309877015935e-6
 1.071254118692619e-6
 3.7538602161096565e-7
 2.1120262864418662e-7
 9.941920943259009e-8
 5.673286879095673e-8
 2.160730842419555e-8

The rank of `ν` is clearly higher than 1, closer to 3:

In [12]:
svdvals(Matrix(ν.Q))

15-element Vector{Float64}:
 5.074213473438938
 1.5645319761479553
 0.9597359871679573
 0.0017398146185114327
 0.0005408264077238202
 0.00012333305842220714
 0.000112117003942734
 8.344767987724291e-6
 1.1444727519464206e-6
 3.1620324683887253e-7
 1.486674010879845e-7
 7.588328515192136e-8
 1.2493207229285292e-8
 4.7632154656602895e-9
 4.013154592036812e-10

Even if the flatness property is not satisfied, we can
still try extracting the minimizer with a low rank decomposition of rank 3.
We find the optimal solution again doing so:

In [13]:
atomic_measure(ν, FixedRank(3))

Atomic measure on the variables x[1], x[2] with 1 atoms:
 at [-1.1555204794988076e-6, -0.9999987643852531] with weight 1.0180428801820578

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*